In [287]:
from deep_translator import GoogleTranslator
import spacy
import services.influencers_sevice as influencers_service
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
import json 
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#visualisation 
import pyLDAvis
import pyLDAvis.gensim

In [288]:
from pymongo import MongoClient
client = MongoClient("localhost", 27017)
db = client.InfluencersMarketing
collection = db.Influencers
posts = db.posts_8

In [289]:
translator = GoogleTranslator(source='auto', target='en')
influencers = influencers_service.get_influencers()
posts=influencers_service.get_posts()
stopwords_en = stopwords.words('english')

In [290]:
def translate_post(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        try:
            translated_content = translator.translate(content)
            print("after tanslation", translated_content)
            influencers_service.update_post(post, key,translated_content)
    
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this caption.")

In [291]:
def translate_hashtags(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        translated_hashtags = []
        for hashtag in hashtags:
            try:
                translated_hashtag = translator.translate(hashtag)
                print(translated_hashtag)
                translated_hashtags.append(translated_hashtag)
                post['hashtags'] = translated_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [292]:
def lemmatize_text(text):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    allowed_postags = ["NOUN", "AD", "VERB", "ADV"]
    doc = nlp(text)
    new_text = []
    for word in doc:
        print(word)
        new_text.append(word.lemma_)
    lemmatized_text = " ".join(new_text)
    return lemmatized_text

In [293]:
def lemmatize_post(posts,key):
    for post in posts: 
        try:
            content = post.get(key)
            lemmatized_content = lemmatize_text(content)
            influencers_service.update_post(post, key , lemmatized_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this hashtag.")

In [294]:
def lemmatize_hashtags(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        lemmatized_hashtags = []
        for hashtag in hashtags:
            try:
                lemmatized_hashtag = lemmatize_text(hashtag)
                lemmatized_hashtags.append(lemmatized_hashtag )
                post['hashtags'] = lemmatized_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [295]:
def remove_emojis(text):
    if(text):
        print("before emoji removal :",text)
        return unidecode(text)
    return text


In [296]:
def remove_hashtags_emojis(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        for hashtag in hashtags:
            try:
                updated_hashtag = remove_emojis(hashtag)
                print(updated_hashtag)
                updated_hashtags .append(updated_hashtags)
                post['hashtags'] = updated_hashtags
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [297]:
def remove_post_emojis(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        no_emoji_content = remove_emojis(content)
        influencers_service.update_post(post, key, no_emoji_content)

In [298]:
def remove_stopwords(text, stopwords_list):
    words = text.split()
    words = [word for word in words if word not in stopwords_list]
    return ' '.join(words)

In [299]:
def remove_post_stopwords(posts,key):
    for post in posts:
        content = post.get(key)
        try:
            no_stopwords_content = remove_stopwords(content, stopwords_en)
            print(no_stopwords_content)
            influencers_service.update_post(post, key, no_stopwords_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this post")

In [300]:
def remove_hashtags_stop_words(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        for hashtag in hashtags:
            try:
                print("before:",hashtag)
                updated_hashtag = remove_stopwords(hashtag)
                print("after:",updated_hashtag)
                updated_hashtags .append(updated_hashtag)
                post['hashtags'] = updated_hashtags
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [301]:
# translate_post(posts,'firstComment')
# translate_post(posts, 'caption')
# translate_hashtags(posts)

In [302]:
# remove_post_emojis(posts, 'firstComment')
# remove_post_emojis(posts, 'caption')
# # remove_hashtags_emojis(posts)

In [303]:
# remove_post_stopwords(posts, 'firstComment')
# remove_post_stopwords(posts, 'caption')
# remove_hashtags_stop_words(posts)

In [304]:
# lemmatize_post(posts,'firstComment')
# lemmatize_post(posts, 'caption')
# lemmatize_hashtags(posts)        

In [305]:
# print("doooooooooone")

Merging the captions and first comment of all posts in one text

In [306]:
def merge_content_posts(posts) : 
    #concatenation of caption and firstComment of all selected posts
    merged_content = ""
    for post in posts:
         #extract caption text
         caption = post.get('caption')
         #extract firstComment ext
         firstComment = post.get('firstComment', [])
         #concatenate caption and firstComment in one string
         if (caption and firstComment):
            merged_content += caption + " " + firstComment
    return merged_content

Extracting words from merged content

In [307]:
def gen_words(text):
    words=[]
    new = gensim.utils.simple_preprocess(text, deacc=True)
    words.append(new)
    return words


In [308]:
def gen_bow(words):
    id2word = corpora.Dictionary(words)
    corpus=[]
    for word in words: 
        new = id2word.doc2bow(word)
        corpus.append(new)
    return  corpus

In [309]:
merge=merge_content_posts(posts)
# print(merge)
words=gen_words(merge)
# print(words)
bow=gen_bow(words)
# print(bow)